In [1]:
%matplotlib notebook
#from selenium import webdriver
import requests
import pandas as pd
from pandas import Timestamp
from pandas.tseries.offsets import DateOffset
import numpy as np
from StringIO import StringIO
import csv
import os
from dateutil import parser
import datetime as dt

FDIR=os.getenv('FDIR')
assert FDIR is not None




In [ ]:
# Find value investment examples:
#    Stock price rose over a period of less than 6 months by more than 20% and held there for 12 months or longer

# Crawl stocks
#    load stock exchange company lists: loadOrGetExchange('NASDAQ')
#    iterate through list of companies on each stock echange

<h2>Run asset allocation experiement</h2>

In [2]:
rs=["FFFGX"]
rs={"FRESX":{'alloc':.2,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FEMKX":{'alloc':.05,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FBGRX":{'alloc':.3,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FIGFX":{'alloc':.15,'min':2500, 'sharesOwned':0, 'amountSpent':0}}

loadSymbols(rs)
for symbol,pkg in rs.items():
    print "{}: {}".format(symbol, len(pkg['data']))
startDate=max([md['data'].index.min() for md in rs.values()])
print "Earliest data for equities is {}".format(startDate)
months=pd.bdate_range(startDate, dt.date.today(), freq='BMS')
    
#TODO: need to assert we have data for each of these symbols or can run simulation

## Any two of the following three can be specified but not all three
## for now we will assume purchasePerMonth and monthsToBalance are the two specified
## purchasePerMonth is the number of purchases that can be made each month
## monthsToBalance is the number of months by which we want to have reached balanced allocation
purchasePerMonth=3; purchaseDays=[7,15,22]; maxPurchasePerMonth=None; monthsToBalance=3; 
baseInvestPerMonth=1000; balanceInvestPerMonth=0; purchasesTilBalance=0;
assert maxPurchasePerMonth is None and (purchasePerMonth is not None or purchaseDays is not None) and monthsToBalance is not None

## totalAtBalance is the smallest 100% total investment at which each asset class can meet its allocation %
#totalAtBalance=max([(1.0/v['alloc'])*v['min'] for s,v in rs.items()])

## remianingToBalance is the amount this asset class needs to still purchase in order to reach its allocation %
#for s,v in rs.items():
#    v['remianingToBalance']=totalAtBalance*(1.0-v['alloc'])

## amountAtEachPurchaseToBalance is the amount of this equity that needs to be purchased at each interval to reach
## the balanced allocation in the set amount of time
#for s,v in rs.items():
#    v['amountAtEachPurchaseToBalance']=totalAtBalance*(1.0-v['alloc'])

buyIntoEquities(rs, startDate)
purchaseAlloc={equity:item['alloc'] for equity, item in rs.items()}
for month in months:
    yearStart = Timestamp(month).is_year_start
    purchaseDates = [month + DateOffset(days=d) for d in purchaseDays]
    for day in purchaseDates:
        print "Buying for month {} and day {}, is year start: {}".format(month, day, yearStart)
        if dateExists(rs, day):
            if yearStart:
                purchaseAlloc=getPurchaseAlloc(rs, day)
                yearStart=False
            for equity, item in rs.items():
                amount=purchaseAlloc[equity]*float(baseInvestPerMonth)
                print "\t {} => alloc: {} amount: {}".format(equity, purchaseAlloc[equity], amount)
                purchaseEquity(item, day, amount)
        else:
            print "\t That day did not exist in all equities"

## Need to include bond payouts into assets
#downloadSymbols(rs.keys())



Found 5 files, loading symbols: FBGRX, FRESX, FEMKX, FIGFX
Loading all symbols...
FBGRX: 2583
FRESX: 2583
FEMKX: 2583
FIGFX: 2121
Earliest data for equities is 2007-11-02 00:00:00
Buying for month 2007-12-03 00:00:00 and day 2007-12-10 00:00:00, is year start: False
	 FBGRX => alloc: 0.3 amount: 300.0
	 FRESX => alloc: 0.2 amount: 200.0
	 FEMKX => alloc: 0.05 amount: 50.0
	 FIGFX => alloc: 0.15 amount: 150.0
Buying for month 2007-12-03 00:00:00 and day 2007-12-18 00:00:00, is year start: False
	 FBGRX => alloc: 0.3 amount: 300.0
	 FRESX => alloc: 0.2 amount: 200.0
	 FEMKX => alloc: 0.05 amount: 50.0
	 FIGFX => alloc: 0.15 amount: 150.0
Buying for month 2007-12-03 00:00:00 and day 2007-12-25 00:00:00, is year start: False
	 That day did not exist in all equities
Buying for month 2008-01-01 00:00:00 and day 2008-01-08 00:00:00, is year start: True
----exceedAllocPercent: 0.4, distributelloc: 0.4
----Total purchase Alloc percent: 0.7
----Equities exceeding alloc: ['FRESX', 'FEMKX', 'FIGFX

AssertionError: 